<a href="https://colab.research.google.com/github/fiyoriyD/csci-164/blob/main/problemSolvingw_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import heapq
import pandas as pd
import time



In [ ]:
#Puzzle Mechanics — Neighbors & Random Walk
def get_neighbors(state, size):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, size)
    possible_moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]

    for dr, dc in possible_moves:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < size and 0 <= new_col < size:
            new_index = new_row * size + new_col
            new_state = list(state)
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            neighbors.append(tuple(new_state))
    return neighbors


def random_walk(initial_state, steps, size):
    current_state = list(initial_state)
    for _ in range(steps):
        neighbors = get_neighbors(current_state, size)
        if neighbors:
            current_state = random.choice(neighbors)
    return tuple(current_state)


In [ ]:
#Heuristic Functions
def out_of_place(state, goal, size):
    return sum(1 for a, b in zip(state, goal) if a != b and a != 0)


def manhattan_distance(state, goal, size):
    distance = 0
    for i in range(size * size):
        if state[i] != 0:
            goal_index = goal.index(state[i])
            row_diff = abs(i // size - goal_index // size)
            col_diff = abs(i % size - goal_index % size)
            distance += row_diff + col_diff
    return distance


In [ ]:
# prompt: Heuristic Functions

#Heuristic Functions (Existing code)
def out_of_place(state, goal, size):
    return sum(1 for a, b in zip(state, goal) if a != b and a != 0)


def manhattan_distance(state, goal, size):
    distance = 0
    for i in range(size * size):
        if state[i] != 0:
            goal_index = goal.index(state[i])
            row_diff = abs(i // size - goal_index // size)
            col_diff = abs(i % size - goal_index % size)
            distance += row_diff + col_diff
    return distance


In [30]:
#Search Function (BFS & A)*
def solve_puzzle(initial_state, goal_state, size, heuristic, algorithm):
    if algorithm == 'bfs':
        queue = [(initial_state, [initial_state], 0)]
        visited = {initial_state}
    elif algorithm == 'astar':
        queue = [(heuristic(initial_state, goal_state, size), initial_state, [initial_state], 0)]
        visited = {initial_state}
    else:
        return None, None, None, None

    nodes_expanded = 0
    start_time = time.time()

    while queue:
        if algorithm == 'bfs':
            current_state, path, cost = queue.pop(0)
        elif algorithm == 'astar':
            _, current_state, path, cost = heapq.heappop(queue)

        nodes_expanded += 1

        if current_state == goal_state:
            end_time = time.time()
            return path, len(path) - 1, nodes_expanded, end_time - start_time

        for neighbor in get_neighbors(current_state, size):
            if neighbor not in visited:
                visited.add(neighbor)
                if algorithm == 'bfs':
                    queue.append((neighbor, path + [neighbor], cost + 1))
                elif algorithm == 'astar':
                    priority = cost + 1 + heuristic(neighbor, goal_state, size)
                    heapq.heappush(queue, (priority, neighbor, path + [neighbor], cost + 1))

    return None, None, None, None


In [31]:
#Set Parameters and Run Experiments
sizes = [3, 4]
goal_states = {
    3: tuple(range(1, 9)) + (0,),
    4: tuple(range(1, 16)) + (0,)
}
initial_states = {
    size: tuple(range(1, size * size)) + (0,) for size in sizes
}
random_walks = [5, 10, 20, 40, 80]
algorithms = ["bfs", "astar"]
heuristics = [out_of_place, manhattan_distance]

results = []


In [32]:
#Generate Problems and Record Results
# Optimized loop to reduce runtime
for size in sizes:
    print(f"\nRunning experiments for {size}x{size} puzzle")

    # Adjust walk depth for 4x4
    rw_set = [5, 10, 20] if size == 4 else [5, 10, 20, 40, 80]

    for rw in rw_set:
        for i in range(2):  # Reduced to 2 problems per walk length for speed
            init = random_walk(initial_states[size], rw, size)

            for algo in algorithms:
                # Skip BFS for 4x4
                if size == 4 and algo == 'bfs':
                    continue

                for heur in heuristics:
                    if algo == 'bfs' and heur != out_of_place:
                        continue  # BFS only with out_of_place

                    print(f"\n{algo.upper()} | {heur.__name__} | {size}x{size} | RW={rw} | Ex {i+1}")
                    start = time.time()

                    path, length, expanded, runtime = solve_puzzle(
                        init, goal_states[size], size, heur, algo
                    )

                    elapsed = round(time.time() - start, 2)
                    print(f"Done in {elapsed}s | Length: {length} | Nodes: {expanded}")

                    results.append({
                        "Puzzle Size": f"{size}x{size}",
                        "Random Walk": rw,
                        "Start State": init,
                        "Algorithm": algo.upper(),
                        "Heuristic": heur.__name__,
                        "Solution Length": length,
                        "Nodes Expanded": expanded,
                        "Runtime (s)": elapsed
                    })




Running experiments for 3x3 puzzle

BFS | out_of_place | 3x3 | RW=5 | Ex 1
Done in 0.0s | Length: 3 | Nodes: 16

ASTAR | out_of_place | 3x3 | RW=5 | Ex 1
Done in 0.0s | Length: 3 | Nodes: 4

ASTAR | manhattan_distance | 3x3 | RW=5 | Ex 1
Done in 0.0s | Length: 3 | Nodes: 4

BFS | out_of_place | 3x3 | RW=5 | Ex 2
Done in 0.0s | Length: 1 | Nodes: 2

ASTAR | out_of_place | 3x3 | RW=5 | Ex 2
Done in 0.0s | Length: 1 | Nodes: 2

ASTAR | manhattan_distance | 3x3 | RW=5 | Ex 2
Done in 0.0s | Length: 1 | Nodes: 2

BFS | out_of_place | 3x3 | RW=10 | Ex 1
Done in 0.0s | Length: 6 | Nodes: 100

ASTAR | out_of_place | 3x3 | RW=10 | Ex 1
Done in 0.0s | Length: 6 | Nodes: 10

ASTAR | manhattan_distance | 3x3 | RW=10 | Ex 1
Done in 0.0s | Length: 6 | Nodes: 7

BFS | out_of_place | 3x3 | RW=10 | Ex 2
Done in 0.0s | Length: 2 | Nodes: 10

ASTAR | out_of_place | 3x3 | RW=10 | Ex 2
Done in 0.0s | Length: 2 | Nodes: 3

ASTAR | manhattan_distance | 3x3 | RW=10 | Ex 2
Done in 0.0s | Length: 2 | Nodes: 3



In [ ]:
#View Results in DataFrame
df = pd.DataFrame(results)
df.sort_values(by=["Puzzle Size", "Random Walk", "Algorithm", "Heuristic"], inplace=True)
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import display
display(df)


,Puzzle Size,Random Walk,Start State,Algorithm,Heuristic,Solution Length,Nodes Expanded,Runtime (s)
2,3x3,5,"(1, 5, 2, 4, 3, 0, 7, 8, 6)",ASTAR,manhattan_distance,5,6,0.000
5,3x3,5,"(1, 2, 3, 4, 5, 6, 7, 0, 8)",ASTAR,manhattan_distance,1,2,0.000
8,3x3,5,"(1, 0, 3, 4, 2, 6, 7, 5, 8)",ASTAR,manhattan_distance,3,4,0.000
83,3x3,5,"(1, 0, 3, 4, 2, 5, 7, 8, 6)",ASTAR,manhattan_distance,3,4,0.000
86,3x3,5,"(1, 2, 3, 4, 5, 6, 7, 0, 8)",ASTAR,manhattan_distance,1,2,0.000
...,...,...,...,...,...,...,...,...
76,4x4,40,"(6, 1, 3, 4, 2, 7, 10, 8, 5, 9, 0, 12, 13, 14,...",ASTAR,out_of_place,12,32,0.000
79,4x4,40,"(1, 2, 3, 4, 13, 0, 11, 7, 5, 9, 10, 8, 14, 6,...",ASTAR,out_of_place,16,382,0.004
72,4x4,40,"(0, 1, 4, 7, 5, 2, 6, 3, 9, 10, 11, 8, 13, 14,...",BFS,out_of_place,10,2254,0.006
75,4x4,40,"(6, 1, 3, 4, 2, 7, 10, 8, 5, 9, 0, 12, 13, 14,...",BFS,out_of_place,12,22415,0.106
